In [0]:
import sys
import random
import string
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.optimizers import RMSprop
from keras.models import load_model

In [0]:
# load text

def load_text(filename):
    with open(filename, 'r') as f:
        text = f.read()
    return text

in_filename = 'drive/shakespeare_poems.txt' # Add your own text file here
text = load_text(in_filename)
print(text[:200])


THE SONNETS

by William Shakespeare

From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his mem


In [0]:
chars = sorted(list(set(text)))
print('Number of distinct characters:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Number of distinct characters: 61


In [0]:
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 ':': 9,
 ';': 10,
 '?': 11,
 'A': 12,
 'B': 13,
 'C': 14,
 'D': 15,
 'E': 16,
 'F': 17,
 'G': 18,
 'H': 19,
 'I': 20,
 'J': 21,
 'K': 22,
 'L': 23,
 'M': 24,
 'N': 25,
 'O': 26,
 'P': 27,
 'R': 28,
 'S': 29,
 'T': 30,
 'U': 31,
 'V': 32,
 'W': 33,
 'Y': 34,
 'a': 35,
 'b': 36,
 'c': 37,
 'd': 38,
 'e': 39,
 'f': 40,
 'g': 41,
 'h': 42,
 'i': 43,
 'j': 44,
 'k': 45,
 'l': 46,
 'm': 47,
 'n': 48,
 'o': 49,
 'p': 50,
 'q': 51,
 'r': 52,
 's': 53,
 't': 54,
 'u': 55,
 'v': 56,
 'w': 57,
 'x': 58,
 'y': 59,
 'z': 60}

In [0]:
# cut the text in sequences of maxlen characters

max_len_chars = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - max_len_chars, step):
    sentences.append(text[i: i + max_len_chars])
    next_chars.append(text[i + max_len_chars])
print('nb sequences:', len(sentences))

nb sequences: 31327


In [0]:
sentences

['\nTHE SONNETS\n\nby William Shakespeare\n\nFr',
 'E SONNETS\n\nby William Shakespeare\n\nFrom ',
 'ONNETS\n\nby William Shakespeare\n\nFrom fai',
 'ETS\n\nby William Shakespeare\n\nFrom faires',
 '\n\nby William Shakespeare\n\nFrom fairest c',
 'y William Shakespeare\n\nFrom fairest crea',
 'illiam Shakespeare\n\nFrom fairest creatur',
 'iam Shakespeare\n\nFrom fairest creatures ',
 ' Shakespeare\n\nFrom fairest creatures we ',
 'akespeare\n\nFrom fairest creatures we des',
 'speare\n\nFrom fairest creatures we desire',
 'are\n\nFrom fairest creatures we desire in',
 '\n\nFrom fairest creatures we desire incre',
 'rom fairest creatures we desire increase',
 ' fairest creatures we desire increase,\nT',
 'irest creatures we desire increase,\nThat',
 'st creatures we desire increase,\nThat th',
 'creatures we desire increase,\nThat there',
 'atures we desire increase,\nThat thereby ',
 'res we desire increase,\nThat thereby bea',
 ' we desire increase,\nThat thereby beauty',
 " desire i

In [0]:
# target character for each sequence

next_chars

['o',
 'f',
 'r',
 't',
 'r',
 't',
 'e',
 'w',
 'd',
 'i',
 ' ',
 'c',
 'a',
 ',',
 'h',
 ' ',
 'e',
 'b',
 'b',
 'u',
 "'",
 'r',
 'e',
 'i',
 't',
 'e',
 'r',
 'i',
 '\n',
 't',
 's',
 'h',
 'r',
 'e',
 's',
 'u',
 ' ',
 ' ',
 'm',
 'd',
 'e',
 'e',
 'H',
 ' ',
 'n',
 'r',
 'e',
 ' ',
 'g',
 ' ',
 'a',
 'h',
 ' ',
 'm',
 'y',
 'B',
 ' ',
 'o',
 'c',
 't',
 'c',
 'd',
 'o',
 'h',
 'e',
 'w',
 'b',
 'g',
 ' ',
 'e',
 '\n',
 'e',
 's',
 't',
 ' ',
 'g',
 "'",
 'f',
 'm',
 'w',
 'h',
 'e',
 '-',
 'b',
 'a',
 'i',
 ' ',
 'e',
 '\n',
 'k',
 'g',
 ' ',
 'm',
 'e',
 'h',
 'e',
 'b',
 'd',
 'c',
 'l',
 's',
 'T',
 ' ',
 'l',
 't',
 ' ',
 'e',
 't',
 't',
 ' ',
 'e',
 ' ',
 'l',
 't',
 ' ',
 'u',
 ':',
 'h',
 ' ',
 'a',
 'a',
 ' ',
 'w',
 'h',
 'w',
 'l',
 's',
 'r',
 'h',
 'r',
 'm',
 't',
 'A',
 ' ',
 'l',
 'h',
 'a',
 ' ',
 ' ',
 'e',
 'a',
 'y',
 'p',
 'n',
 '\n',
 't',
 'n',
 'h',
 'e',
 'w',
 'b',
 ' ',
 'r',
 's',
 't',
 ' ',
 'n',
 'n',
 '\n',
 'd',
 'e',
 'e',
 'c',
 'r',
 'm',
 "'",

In [0]:
# Vectorization

x = np.zeros((len(sentences), max_len_chars, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [0]:
# build the model using GRU

print('Build model...')
model = Sequential()
model.add(GRU(128, input_shape=(max_len_chars, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
model.fit(x, y,batch_size=128,epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
31327/31327 [==============================] - 57s 2ms/step - loss: 2.2682
Epoch 2/10
31327/31327 [==============================] - 56s 2ms/step - loss: 1.9105
Epoch 3/10
31327/31327 [==============================] - 56s 2ms/step - loss: 1.7677
Epoch 4/10
31327/31327 [==============================] - 56s 2ms/step - loss: 1.6734
Epoch 5/10
31327/31327 [==============================] - 55s 2ms/step - loss: 1.5973
Epoch 6/10
31327/31327 [==============================] - 48s 2ms/step - loss: 1.5403
Epoch 7/10
31327/31327 [==============================] - 27s 876us/step - loss: 1.5036
Epoch 8/10
31327/31327 [==============================] - 27s 877us/step - loss: 1.4713
Epoch 9/10
31327/31327 [==============================] - 27s 874us/step - loss: 1.4490
Epoch 10/10
31327/31327 [==============================] - 27s 876us/step - loss: 1.4257


In [0]:
model.save("poem_gen_model.h5")

# Inference

In [0]:
from keras.models import load_model
model_loaded = load_model('poem_gen_model.h5')

In [0]:
def generate_poem(model, num_chars_to_generate=400):
    start_index = random.randint(0, len(text) - max_len_chars - 1)
    generated = ''
    sentence = text[start_index: start_index + max_len_chars]
    generated += sentence
    print("Seed sentence: {}".format(generated))
    for i in range(num_chars_to_generate):
        x_pred = np.zeros((1, max_len_chars, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 1)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [0]:
generate_poem(model_loaded, 100)

Seed sentence:  to his subject lends not some small glo


" to his subject lends not some small gloen.\nI\n'Wiss hume thun my be thum I part molinFor minr,\nThough st yealt of thy foedbere imy giths:\nAn"

In [0]:
generate_poem(model_loaded, 100)

Seed sentence: pretty looks have been mine enemies,
And


'pretty looks have been mine enemies,\nAnd summmmmite it Time swill hold love and ust.\nAnd thou heart whereferayed me henule,\nThat which have,'